# Assignment 3

**Name:** Milagros Contreras Elizalde

**e-mail:** -- milagros.contreras8430@alumnos.udg.mx --

In [49]:
!pip install dash

In [50]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from scipy.stats import cauchy
from plotly.subplots import make_subplots

We define the Vec2d class:

In [51]:
class Vec2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vec2d(self.x + other.x, self.y + other.y)

    def __mul__(self, scalar):
        return Vec2d(self.x * scalar, self.y * scalar)

# Brownian Motion

Brownian motion is the random motion of particles suspended in a medium.


In [52]:
# Functions to generate trajectories
def brownian_motion(steps, speed, start_pos):
    traj = [Vec2d(*start_pos)]
    for _ in range(steps):
        angle = np.random.uniform(0, 2 * np.pi)
        step = Vec2d(np.cos(angle), np.sin(angle)) * speed
        traj.append(traj[-1] + step)
    return traj

This function simulates Brownian motion by generating a random path for a particle. It starts from a given position, then repeatedly moves the particle in random directions for a specified number of steps.

# Random Walk

A random walk is known as a random process which describes a path including a succession of random steps in the mathematical space

In [53]:
def correlated_random_walk(steps, speed, start_pos, cauchy_coeff):
    traj = [Vec2d(*start_pos)]
    angle = 0 
    for _ in range(steps):
        angle += cauchy.rvs(scale=cauchy_coeff)  
        step = Vec2d(np.cos(angle), np.sin(angle)) * speed
        traj.append(traj[-1] + step)
    return traj

This function simulates a "correlated random walk," where the direction of movement changes gradually based on random increments.The cauchy_coeff controls how much the direction can change between steps.

# Lévy flight

 Lévy flight is a random walk in which the step-lengths have a stable distribution, a probability distribution that is heavy-tailed. When defined as a walk in a space of dimension greater than one, the steps made are in isotropic random directions. Later researchers have extended the use of the term "Lévy flight" to also include cases where the random walk takes place on a discrete grid rather than on a continuous space.

In [54]:
def levy_flight(steps, speed, start_pos, cauchy_coeff, alpha):
    traj = [Vec2d(*start_pos)]
    for _ in range(steps):
        direction = np.random.uniform(0, 2 * np.pi)
        step_length = (np.random.pareto(alpha) + 1) * cauchy_coeff 
        step = Vec2d(np.cos(direction), np.sin(direction)) * step_length * speed
        traj.append(traj[-1] + step)
    return traj


This function simulates a Lévy flight, where a particle moves in random directions with mostly short steps but sometimes makes longer, unpredictable jumps. The step lengths follow a Pareto distribution, controlled by the alpha parameter, creating a mix of frequent small movements and rare long ones.

# Functions to calculate metrics

**Path length**


This function calculates the total length of a path by summing the distances between consecutive points. It uses the Euclidean distance formula to compute these distances.

In [55]:
def calculate_path_length(trajectory):
    length = 0
    for i in range(1, len(trajectory)):
        prev, curr = trajectory[i-1], trajectory[i]
        length += np.sqrt((curr.x - prev.x)**2 + (curr.y - prev.y)**2)
    return length


**Mean squared displacement**

In statistical mechanics, the mean squared displacement (MSD, also mean square displacement, average squared displacement, or mean square fluctuation) is a measure of the deviation of the position of a particle with respect to a reference position over time. It is the most common measure of the spatial extent of random motion, and can be thought of as measuring the portion of the system "explored" by the random walker. 

In [56]:
def calculate_msd(trajectory):
    msd = []
    for tau in range(1, len(trajectory)):
        displacements = [(trajectory[i + tau].x - trajectory[i].x)**2 + (trajectory[i + tau].y - trajectory[i].y)**2 
                        for i in range(len(trajectory) - tau)]
        msd.append(np.mean(displacements))
    return msd

This function, calculate_msd, calculates the Mean Squared Displacement (MSD) for a given trajectory. MSD is a measure used in physics to quantify how much a particle's position changes over time.

References
- Wikipedia contributors. (2024, September 27). Brownian motion. Wikipedia. https://en.wikipedia.org/wiki/Brownian_motion
- Random Walks: A Review of Algorithms and Applications. (s. f.). IEEE TRANSACTIONS ON EMERGING TOPICS IN COMPUTATIONAL INTELLIGENCE, VOL. 0, https://arxiv.org/pdf/2008.03639#:~:text=Abstract%E2%80%94A%20random%20walk%20is,as%20mathematics%20and%20computer%20science.
- Wikipedia contributors. (2024, April 25). Lévy flight. Wikipedia. https://en.wikipedia.org/wiki/L%C3%A9vy_flight